# Brain MRI Metastasis Segmentation Assignment

### Step 1: Data Preprocessing

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Load MRI images and corresponding masks
def load_data(image_dir, mask_dir):
    images = []
    masks = []
    for img_name in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_name)
        mask_path = os.path.join(mask_dir, img_name)  # Assuming masks have the same names

        # Skip images without masks or vice versa
        if os.path.exists(mask_path):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # CLAHE for enhancing image contrast
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            img = clahe.apply(img)

            images.append(img)
            masks.append(mask)
    
    return np.array(images), np.array(masks)

# Normalization and augmentation (simplified)
def normalize_augment(images, masks):
    images = images / 255.0  # Normalize to range [0, 1]
    # Data augmentation (rotation, flipping, etc.) can be applied here if necessary
    return images, masks

# Load and split dataset
image_dir = '/content/images'
mask_dir = '/content/masks'
images, masks = load_data(image_dir, mask_dir)
images, masks = normalize_augment(images, masks)
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2)


### Step 2: Model Implementation (Nested U-Net and Attention U-Net)

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models

def inceptionv3_unet(input_size=(256, 256, 3)):
    # Load pre-trained InceptionV3, excluding the top layer
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_size)

    # Freeze the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    inputs = layers.Input(shape=input_size)
    
    # Use InceptionV3 as the encoder part of U-Net
    x = base_model(inputs)
    
    # Add U-Net decoder part here (up-sampling layers)
    conv1 = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    up1 = layers.UpSampling2D(size=(2, 2))(conv1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(up1)
    up2 = layers.UpSampling2D(size=(2, 2))(conv2)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(up2)
    up3 = layers.UpSampling2D(size=(2, 2))(conv3)
    
    # Final convolution to get a single channel for segmentation
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(up3)
    
    model = models.Model(inputs, outputs)
    return model

# Instantiate the InceptionV3-based U-Net
inception_unet_model = inceptionv3_unet()
inception_unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


### Step 3: Model Training and Evaluation

In [ ]:
from sklearn.metrics import jaccard_score

def dice_score(y_true, y_pred):
    smooth = 1.0
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

# Train U-Net++
unetpp_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate U-Net++
y_pred = unetpp_model.predict(X_test)
dice_unetpp = dice_score(y_test, y_pred)

# Train Attention U-Net
att_unet_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate Attention U-Net
y_pred_att = att_unet_model.predict(X_test)
dice_att = dice_score(y_test, y_pred_att)

# Compare the models
print(f"Nested U-Net DICE Score: {dice_unetpp}")
print(f"Attention U-Net DICE Score: {dice_att}")


### Step 4: Web Application (FAST API Backend & Streamlit UI)

In [ ]:
# FAST API Backend Code

from fastapi import FastAPI, UploadFile
import numpy as np
from PIL import Image

app = FastAPI()

@app.post("/predict/")
async def predict(file: UploadFile):
    image = np.array(Image.open(file.file).convert('L'))
    # Preprocess and predict using your trained model
    prediction = unetpp_model.predict(image)  # Replace with your model of choice
    return {"segmentation": prediction.tolist()}


In [ ]:
# Streamlit UI Code

import streamlit as st
import requests
from PIL import Image

st.title("Brain MRI Metastasis Segmentation")
uploaded_file = st.file_uploader("Upload a Brain MRI Image", type=["png", "jpg", "jpeg"])

if uploaded_file is not None:
    img = Image.open(uploaded_file)
    st.image(img, caption="Uploaded MRI.", use_column_width=True)
    
    # Send image to backend API for segmentation
    response = requests.post("http://localhost:8000/predict/", files={"file": uploaded_file})
    if response.status_code == 200:
        segmentation = response.json()["segmentation"]
        st.image(segmentation, caption="Metastasis Segmentation", use_column_width=True)
